In [1162]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [1163]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [1164]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];
for ram_pos in 1:L
    shift_bit = shift_1_3_bits[ram_phy[ram_pos]]
    println(shift_bit)
end

[1, 2, 3, 4]
[1, 4, 2, 3]
[1, 3, 4, 2]
0
1
1
0


In [1165]:
initial_state = productMPS(qubit_site, [1,1,1,1])
include("global_adder_passthrough.jl")
global_adder_folded(initial_state, ram_phy, shift_1_3_bits, qubit_site)
# single_site_add_folded!(shift_1_3_bits[ram_phy[ram_pos]], ram_pos, initial_state)


LoadError: BoundsError: attempt to access Tuple{} at index [1]

In [ ]:
tensor = contract(initial_state)
println(inds(tensor))
print_nonzero_coordinates(tensor.tensor)

((dim=2|id=371|"Qubit,Site,n=1"), (dim=2|id=798|"Qubit,Site,n=2"), (dim=2|id=159|"Qubit,Site,n=3"), (dim=2|id=180|"Qubit,Site,n=4"), (dim=2|id=983|"Carry,c=4"), (dim=2|id=671|"Carry,c=5"), (dim=2|id=56|"Qubit,Site,n=5"), (dim=2|id=550|"Qubit,Site,n=6"), (dim=2|id=484|"Qubit,Site,n=7"), (dim=2|id=436|"Qubit,Site,n=8"))
Nonzero at coordinates (2, 2, 1, 1, 1, 1, 1, 1, 1, 2): -1.5597960245906893e-17
Nonzero at coordinates (1, 2, 2, 1, 1, 1, 1, 1, 1, 2): 0.9999999999999999
Nonzero at coordinates (2, 2, 1, 1, 2, 1, 1, 1, 1, 2): 0.9999999999999998
Nonzero at coordinates (1, 2, 2, 1, 2, 1, 1, 1, 1, 2): 1.2169493844099183e-16


In [ ]:
function add_8bit_simple(binary_str::String)
    input_num = parse(Int, lpad(binary_str, 8, '0')[end-7:end], base=2)
    result = (input_num + 0b01010101) & 0xFF
    return lpad(string(result, base=2), 8, '0')
end

add_8bit_simple("00000001")

"01010110"

In [ ]:
function perm(bit_str::String, pattern::Vector{Int})
    permuted = ['0' for _ in 1:8]
    for (i, pos) in enumerate(pattern)
        permuted[pos] = bit_str[i]
    end
    return join(permuted)
end

perm(add_8bit_simple("00000001"), [1, 3, 5, 7, 8, 6, 4, 2])

"00110110"

In [ ]:
for i in 0:255
    input_str_unfolded = lpad(string(i, base=2), 8, '0')
    input_str_folded = perm(input_str_unfolded, phy_ram)
    output_str_unfolded = add_8bit_simple(input_str_unfolded)
    output_str_folded = perm(output_str_unfolded, phy_ram)
    println(input_str_folded, " ", output_str_folded)
end

00000000 01100110
01000000 00110110
00010000 01110110
01010000 00100011
00000100 01100011
01000100 00110011
00010100 01110011
01010100 00100111
00000001 01100111
01000001 00110111
00010001 01110111
01010001 00101000
00000101 01101000
01000101 00111000
00010101 01111000
01010101 00101100
00000010 01101100
01000010 00111100
00010010 01111100
01010010 00101001
00000110 01101001
01000110 00111001
00010110 01111001
01010110 00101101
00000011 01101101
01000011 00111101
00010011 01111101
01010011 00101010
00000111 01101010
01000111 00111010
00010111 01111010
01010111 00101110
00001000 01101110
01001000 00111110
00011000 01111110
01011000 00101011
00001100 01101011
01001100 00111011
00011100 01111011
01011100 00101111
00001001 01101111
01001001 00111111
00011001 01111111
01011001 10000000
00001101 11000000
01001101 10010000
00011101 11010000
01011101 10000100
00001010 11000100
01001010 10010100
00011010 11010100
01011010 10000001
00001110 11000001
01001110 10010001
00011110 11010001
01011110 1

In [ ]:
using Random
using ITensors
i = Index(2, "Carry,c=1")
j = Index(2, "Carry,c=2")
A = randomITensor(i, j)
B = randomITensor(j, i)
pair = (A, B)

(ITensor ord=2
Dim 1: (dim=2|id=354|"Carry,c=1")
Dim 2: (dim=2|id=156|"Carry,c=2")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 1.0138643023841494   -0.3407956368420073
 0.20905176626709085  -0.290925084710782, ITensor ord=2
Dim 1: (dim=2|id=156|"Carry,c=2")
Dim 2: (dim=2|id=354|"Carry,c=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.02267049826257342  0.9366063913429162
 0.3763027114359013   0.6149718056079265)

In [ ]:
reverse(pair)

(ITensor ord=2
Dim 1: (dim=2|id=156|"Carry,c=2")
Dim 2: (dim=2|id=354|"Carry,c=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.02267049826257342  0.9366063913429162
 0.3763027114359013   0.6149718056079265, ITensor ord=2
Dim 1: (dim=2|id=354|"Carry,c=1")
Dim 2: (dim=2|id=156|"Carry,c=2")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 1.0138643023841494   -0.3407956368420073
 0.20905176626709085  -0.290925084710782)

In [ ]:
inds(initial_state[1])
orthogonalize!(initial_state, 1)

MPS
[1] ((dim=2|id=371|"Qubit,Site,n=1"), (dim=2|id=882|"Link,l=1"))
[2] ((dim=2|id=798|"Qubit,Site,n=2"), (dim=4|id=784|"Link,l=2"), (dim=2|id=882|"Link,l=1"))
[3] ((dim=2|id=159|"Qubit,Site,n=3"), (dim=4|id=291|"Link,l=3"), (dim=4|id=784|"Link,l=2"))
[4] ((dim=2|id=180|"Qubit,Site,n=4"), (dim=2|id=983|"Carry,c=4"), (dim=1|id=341|"Link,l=4"), (dim=4|id=291|"Link,l=3"))
[5] ((dim=2|id=671|"Carry,c=5"), (dim=2|id=56|"Qubit,Site,n=5"), (dim=1|id=125|"Link,l=5"), (dim=1|id=341|"Link,l=4"))
[6] ((dim=2|id=550|"Qubit,Site,n=6"), (dim=1|id=386|"Link,l=6"), (dim=1|id=125|"Link,l=5"))
[7] ((dim=2|id=484|"Qubit,Site,n=7"), (dim=1|id=633|"Link,l=7"), (dim=1|id=386|"Link,l=6"))
[8] ((dim=2|id=436|"Qubit,Site,n=8"), (dim=1|id=633|"Link,l=7"))


In [ ]:
initial_state[4].tensor

Dim 1: (dim=2|id=180|"Qubit,Site,n=4")
Dim 2: (dim=2|id=983|"Carry,c=4")
Dim 3: (dim=1|id=341|"Link,l=4")
Dim 4: (dim=4|id=291|"Link,l=3")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×1×4
[:, :, 1, 1] =
  0.0                  0.0
 -0.7071067811865476  -0.7071067811865476

[:, :, 1, 2] =
  0.0                 0.0
 -0.7071067811865475  0.7071067811865475

[:, :, 1, 3] =
 -0.9732489894677304     -0.22975292054736118
 -2.220446049250313e-16  -2.220446049250313e-16

[:, :, 1, 4] =
 -0.22975292054736096    0.9732489894677302
  8.326672684688674e-17  1.1102230246251565e-16

In [ ]:
tmp = initial_state[1] * initial_state[2]

ITensor ord=3 (dim=2|id=371|"Qubit,Site,n=1") (dim=2|id=798|"Qubit,Site,n=2") (dim=4|id=784|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
inds(tmp)

((dim=2|id=371|"Qubit,Site,n=1"), (dim=2|id=798|"Qubit,Site,n=2"), (dim=4|id=784|"Link,l=2"))

In [ ]:
tmp.tensor[:,2,:,1,1]

Dim 1: 2
Dim 2: 4
NDTensors.Dense{Float64, Base.ReshapedArray{Float64, 1, SubArray{Float64, 2, Array{Float64, 3}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64, Base.Slice{Base.OneTo{Int64}}}, false}, Tuple{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64}}}}
 2×4
 0.0                 -0.9999999999999999  0.0  0.0
 0.9999999999999998   0.0                 0.0  0.0

In [ ]:
single_site_add_folded!(shift_1_3_bits[ram_phy[2]], 2, initial_state)
println(shift_1_3_bits[ram_phy[2]])

Index{Int64}[(dim=2|id=371|"Qubit,Site,n=1"), (dim=2|id=798|"Qubit,Site,n=2"), (dim=2|id=159|"Qubit,Site,n=3"), (dim=2|id=180|"Qubit,Site,n=4"), (dim=2|id=671|"Carry,c=5"), (dim=2|id=550|"Qubit,Site,n=6"), (dim=2|id=484|"Qubit,Site,n=7"), (dim=2|id=436|"Qubit,Site,n=8")]
1


In [ ]:
lsb_tensor = ITensor(filterinds(initial_state[2], tags="Carry,c=1"))
lsb_tensor[inds(lsb_tensor)[1]=>1] = 1.0
# print(inds(lsb_tensor))
initial_state[2] = initial_state[2] * lsb_tensor;
# println(inds(initial_state[2]))

# discard the carry out of the msb tensor
msb_tensor = ITensor(filterinds(initial_state[1], tags="Carry,c=0"))
msb_tensor[inds(msb_tensor)[1]=>1] = 1.0
msb_tensor[inds(msb_tensor)[1]=>2] = 1.0
initial_state[1] = initial_state[1] * msb_tensor;

LoadError: BoundsError: attempt to access Tuple{} at index [1]

In [ ]:
println(inds(initial_state[2]))

In [ ]:
tmp2 = initial_state[1] * initial_state[2] * initial_state[3]
println(inds(tmp2))

In [ ]:
size(tmp2)

In [ ]:
using ITensors

function print_nonzero_coordinates(T)
    # Get the dimensions of the tensor
    dims = size(T)
    
    # Iterate through all possible indices
    for idx in CartesianIndices(dims)
        # Check if element is nonzero
        if T[idx] != 0
            # Convert CartesianIndex to tuple for cleaner output
            coords = Tuple(idx)
            println("Nonzero at coordinates $coords: $(T[idx])")
        end
    end
end

print_nonzero_coordinates(tmp2.tensor)

In [ ]:
c2 = inds(initial_state[2])[2]
println(inds(initial_state[2]))
c4 = Index(2, "Carry,c=4")
T_Lm1 = create_addition_tensor_with_carry(1, qubit_site[L-1], prime(qubit_site[L-1]), c4, c2)

In [ ]:
tensor = contract(initial_state)
println(inds(tensor))

In [ ]:

# qubit_site = siteinds(initial_state)
# T_left_ind = Index(2, "Carry,c=$(ram_pos-1)")
# T_right_ind = Index(2, "Carry,c=$(ram_pos)")
# T = create_addition_tensor_with_carry(shift_bit, qubit_site[ram_pos], prime(qubit_site[ram_pos]), T_right_ind, T_left_ind);
# id_op_right_ind = Index(2, "Carry,c=$(ram_pos+1)")
# id_op = create_identity_tensor_4d(qubit_site[ram_pos+1], prime(qubit_site[ram_pos+1]), id_op_right_ind, T_right_ind);
# # println("T", inds(T))
# # println("id_op", inds(id_op))
# gate = T * id_op
# # figure out the left and right indices for the SVD
# left_original_inds = inds(initial_state[ram_pos]) # bookkeep some of the original indices from the left site for SVD
# link_right = filterinds(initial_state[ram_pos+1], tags="Link,l=$(ram_pos)")
# left_original_inds = setdiff(left_original_inds, link_right)
# # println("left_original_inds ", left_original_inds)

# # contract the gate with the two qubit sites
# psi = gate * (initial_state[ram_pos] * initial_state[ram_pos+1])
# noprime!(psi)
# carry_left = filterinds(psi, tags="Carry,c=$(ram_pos-1)")
# # println("carry_left", carry_left)
# link_left = filterinds(psi, tags="Link,l=$(ram_pos-1)")
# # println("link_left ", link_left)
# qubit_left = filterinds(psi, tags="Site,n=$(ram_pos)")
# # println(qubit_left)
# left_inds = unioninds(carry_left, link_left, qubit_left, left_original_inds)
# println("left_inds ", left_inds)
# # println(typeof(psi))
# U, S, V = svd(psi, left_inds; cutoff=1e-12, lefttags = "Link,l=$(ram_pos)")
# # println(inds(initial_state))
# # new_state = copy(initial_state)
# initial_state[ram_pos] = U
# initial_state[ram_pos+1] = S*V

In [ ]:
new_psi = initial_state[1] * initial_state[2]
println(inds(new_psi))
println(new_psi.tensor[:,1,1,:,1])

In [ ]:
psi = gate * (initial_state[1] * initial_state[2])

In [ ]:
psi.tensor[:,2,:,1,1]

In [ ]:
println(inds(initial_state[1]))

In [ ]:
typeof(initial_state[1])

In [ ]:
tensor1 = initial_state[1]
indices1 = inds(tensor1)
println(indices1)
link_num = 2
tensor1[indices1[1]=>2, indices1[2]=>2, indices1[3]=>link_num]

In [ ]:
tensor2 = initial_state[2]
indices2= inds(tensor2)
println(indices2)
tensor2[indices2[1] => 1, indices2[2]=>1, indices2[3]=>2, indices2[4]=>1]

In [ ]:
initial_state[2].tensor[2,:,:]

In [ ]:
# initial_state = productMPS(qubit_site, [1,2,2,1])
# shift_bit = 0
# ram_pos = 1
# include("global_adder_passthrough.jl")
# T_left_ind = Index(2, "Carry,c=$(ram_pos-1)")
# T_right_ind = Index(2, "Carry,c=$(ram_pos)")
# T = create_addition_tensor_with_carry(shift_bit, qubit_site[ram_pos], prime(qubit_site[ram_pos]), T_left_ind, T_right_ind);
# id_op_right_ind = Index(2, "Carry,c=$(ram_pos+1)")
# id_op = create_identity_tensor_4d(qubit_site[ram_pos+1], prime(qubit_site[ram_pos+1]), T_right_ind, id_op_right_ind);
# gate = T * id_op
# # figure out the left and right indices for the SVD
# left_original_inds = inds(initial_state[ram_pos]) # bookkeep some of the original indices from the left site for SVD
# link_right = filterinds(initial_state[ram_pos+1], tags="Link,l=$(ram_pos)")
# left_original_inds = setdiff(left_original_inds, link_right)
# # println("left_original_inds ", left_original_inds)

# # contract the gate with the two qubit sites
# psi = gate * (initial_state[ram_pos] * initial_state[ram_pos+1])
# noprime!(psi)
# carry_left = filterinds(psi, tags="Carry,c=$(ram_pos-1)")
# # println("carry_left", carry_left)
# link_left = filterinds(psi, tags="Link,l=$(ram_pos-1)")
# # println("link_left ", link_left)
# qubit_left = filterinds(psi, tags="Site,n=$(ram_pos)")
# # println(qubit_left)
# left_inds = unioninds(carry_left, link_left, qubit_left, left_original_inds)
# # println("left_inds ", left_inds)
# # println(typeof(psi))
# U, S, V = svd(psi, left_inds; cutoff=1e-12, lefttags = "Link,l=$(ram_pos)")
# # println(inds(initial_state))
# # new_state = copy(initial_state)
# initial_state[ram_pos] = U
# initial_state[ram_pos+1] = S*V

In [ ]:
# println(inds(initial_state))

# msb_tensor = ITensor(filterinds(initial_state[1], tags="Carry,c=0"))
# msb_tensor[inds(msb_tensor)[1]=>1] = 1.0
# msb_tensor[inds(msb_tensor)[1]=>2] = 1.0
# initial_state[1] = initial_state[1] * msb_tensor;
# normalize!(initial_state)

# println(initial_state)

In [ ]:
# L = 40
# ancilla = 0
# folded = true
# seed_vec = 123457
# xj = Set([1//3, 2//3])
# i1 = 1
# p_ctrl = 1.0
# p_proj = 0.0
# _maxdim = 50
# _cutoff = 1e-15
# seed = 123457
# x0 = nothing
# qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
# println(phy_list)
# println(ram_phy)
# println(phy_ram)
# rng = MersenneTwister(seed_vec)
# rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# # initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# # println(initial_state)
# shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# # @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L]



In [ ]:
# # Method 1: @time (shows time + memory allocation + GC time)
# @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];

# # Method 2: @allocated (only memory allocation in bytes)
# memory_allocated = @allocated adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];
# println("Memory allocated: $(memory_allocated / 1024^2) MB")

# # Method 3: Manual tracking with GC.gc()
# GC.gc()  # Force garbage collection
# mem_before = Base.gc_live_bytes()
# adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];
# GC.gc()  # Force garbage collection
# mem_after = Base.gc_live_bytes()
# println("Live memory change: $((mem_after - mem_before) / 1024^2) MB")


In [ ]:
# mps = _initialize_vector(L, ancilla, nothing, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim)
# @time apply(adder[3], mps)

In [ ]:
# # Method 6: System-level memory monitoring (Linux/Unix)
# # This tracks actual RSS (Resident Set Size) memory usage

# function get_memory_usage()
#     try
#         # Read from /proc/self/status on Linux
#         status = read("/proc/self/status", String)
#         for line in split(status, '\n')
#             if startswith(line, "VmRSS:")
#                 # Extract memory in kB and convert to MB
#                 memory_kb = parse(Int, split(line)[2])
#                 return memory_kb / 1024  # Convert to MB
#             end
#         end
#     catch
#         # Fallback: use Julia's garbage collector info
#         return Base.gc_live_bytes() / 1024^2
#     end
#     return 0.0
# end

# # Track memory before and after
# println("Memory before: $(get_memory_usage()) MB")
# @time adder=[adder_MPO(i1,xj,qubit_site,L,phy_ram,phy_list) for i1 in 1:L];
# println("Memory after: $(get_memory_usage()) MB")
